In [8]:
import numpy as np
import netket as nk
import functions as f

/home/keisuke/Documents/Research/sshfs/DimerMaster/functions.py:65: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec)


In [9]:
from conf import *
from numba import njit

In [10]:
np.zeros(3, dtype=np.intp)[0:3]
np.random.rand(4)
5.1 //3

1.0

In [11]:
@njit
def foo(x,y):
    print((x//y).astype(np.int64))
    return x

In [12]:
x = np.random.randint(0,100, 10)
y = np.random.randint(1,10, 10)

In [13]:
foo(x,y)

[ 6  4 17  5  7 15  5 10 24  4]


array([55,  8, 51, 31, 57, 95, 36, 54, 73, 33])

In [14]:
h = 1.0
V = 1.0

length = [4, 4]
name = 'h={}V={}l={}'.format(h, V, length)
n_chains = 10
sweep_size = 70
n_samples = 10000



g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))


ma = nk.machine.RbmDimer(hi, hex_, alpha = alpha, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float)

ma.load('save/ma/'+name)


sa = nk.sampler.DimerMetropolisLocal(machine=ma, op=op_transition, length = length, n_chains=n_chains, sweep_size = 70)
sa.generate_samples(10) # discard the begginings of metropolis sampling.
print('discard samples')
samples_state = sa.generate_samples(int(n_samples / n_chains))
samples_state = samples_state.reshape(-1, ma.hilbert.size)

print('prepared initial samples')




4
discard samples
prepared initial samples


In [15]:
local_states = op._local_states
basis = op._basis

In [16]:
samples_state

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ..., -1., -1., -1.],
       [ 1., -1., -1., ..., -1.,  1.,  1.],
       ...,
       [ 1.,  1., -1., ...,  1.,  1.,  1.],
       [-1., -1., -1., ...,  1.,  1.,  1.],
       [-1., -1., -1., ...,  1.,  1.,  1.]])

In [17]:
((samples_state[:, op._acting_on[0]] + 1) / 2 * basis).sum(axis=1)

array([15.,  7.,  7., ..., 15.,  8.,  8.])

In [18]:
def dot_1(x, y):
    return (x * y).sum(axis=1)

@njit
def dot_numba(x,z):
    y = np.zeros(x.shape[0])
    for i in range(x.shape[1]):
        y += x[:,i] * z[i]
    return y 

In [19]:
# %%timeit 
X = (np.random.choice([-1,1], (10000,4)) + 1)/2
i = dot_1(X, basis)

In [20]:
op._n_conns[i.astype(np.int)]

array([[1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [224]:
i

array([5., 6., 3., ..., 0., 6., 7.])

In [ ]:
conn = op._n_conns

In [210]:
# %%timeit 
X = np.random.randint(1,2, (10000,4))
dot_numba(X, basis)

array([15., 15., 15., ..., 15., 15., 15.])

In [109]:
%%timeit 
a = 0
for k in range(acting_size):
    a += (
        _np.searchsorted(local_states, x_i[acting_size- k - 1])
        * basis[k]
    )

8.12 µs ± 218 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [170]:
@njit
def dot(x, acting_on):
    
    xx = x[:, acting_on]
    
    return xx

In [171]:
dot(samples_state, op._acting_on[0])

array([[-1., -1., -1.,  1.],
       [-1., -1.,  1., -1.],
       [-1., -1.,  1., -1.],
       ...,
       [-1., -1., -1.,  1.],
       [ 1., -1., -1.,  1.],
       [-1., -1., -1.,  1.]])